In [30]:
import numpy as np

In [1]:
import pickle

with open('/Users/riccardotedoldi/Desktop/ais/I/sem2/applied.nlp/go_term_encoding/proteins/go_term/dictionary_all_go_tensor_padded.pkl', 'rb') as f:
    go_term = pickle.load(f)

In [6]:
# what does it stand for the BP, CC, MF?
# Biological Process (BP):  	biological_process 
# Cellular Component (CC):  	cellular_component 
# Molecular Function (MF):  	molecular_function 

print('Number of gene annotated with Molecular Function (MF):',len(go_term['MF']))
print('Number of gene annotated with Biological Process (BP):',len(go_term['BP']))
print('Number of gene annotated with Cellular Component (CC):',len(go_term['CC']))

Number of gene annotated with Molecular Function (MF): 18259
Number of gene annotated with Biological Process (BP): 16897
Number of gene annotated with Cellular Component (CC): 17455


Now I am willing to expariments to get some results given those data.

In [ ]:
import torch
import torch.nn as nn

import pytorch_lightning as pl



class GoTermDataset(torch.utils.data.Dataset):
    def __init__(self, go_term, go_term_type):
        self.go_term = go_term[go_term_type]
        self.go_term_type = go_term_type
        self.len = len(go_term[go_term_type])

    def __len__(self):
        return len(self.go_term)
    
    def __getnumberofitems__(self):
        return self.len

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        go_term = self.go_term[idx]
        go_term_type = self.go_term_type[idx]

        return go_term, go_term_type


In [ ]:
import torch

[go_term['MF'][idx] for idx in go_term['MF'].keys()]

gene_id = go_term['MF'].keys()

In [17]:
from genes_ncbi_mus_musculus_proteincoding import GENEID2NT
import pandas as pd

# here we have the table with all the gene id and the relative description
# the accession number and other attributes

# now we would like to retrive the sequence of those genes to enable the 
# model to condition on the go term and produce the sequence of the protein

pd.DataFrame.from_dict(GENEID2NT, orient='index').head()

,tax_id,Org_name,GeneID,CurrentID,Status,Symbol,Aliases,description,other_designations,map_location,chromosome,genomic_nucleotide_accession_version,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,orientation,exon_count,OMIM,no_hdr0
1,9606,Homo sapiens,1,0,live,A1BG,"[A1B, ABG, GAB, HYST2477]",alpha-1-B glycoprotein,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...,19q13.43,19,NC_000019.10,58345183,58353492,minus,8,[138670],
2,9606,Homo sapiens,2,0,live,A2M,"[A2MD, CPAMD5, FWP007, S863-7]",alpha-2-macroglobulin,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...,12p13.31,12,NC_000012.12,9067708,9116229,minus,37,[103950],
9,9606,Homo sapiens,9,0,live,NAT1,"[AAC1, MNAT, NAT-1, NATI]",N-acetyltransferase 1,arylamine N-acetyltransferase 1|N-acetyltransf...,8p22,8,NC_000008.11,18170467,18223689,plus,11,[108345],
10,9606,Homo sapiens,10,0,live,NAT2,"[AAC2, NAT-2, PNAT]",N-acetyltransferase 2,arylamine N-acetyltransferase 2|N-acetyltransf...,8p22,8,NC_000008.11,18386301,18401218,plus,4,[612182],
12,9606,Homo sapiens,12,0,live,SERPINA3,"[AACT, ACT, GIG24, GIG25]",serpin family A member 3,alpha-1-antichymotrypsin|cell growth-inhibitin...,14q32.13,14,NC_000014.9,94612391,94624053,plus,7,[107280],


I tried to download the gene sequence from NCBI using the API... but the problem it seems it takes too long. Thus, I decided to check if I can find the file zip online with inside all the sequences given the gene_id.

In [ ]:
from tqdm import tqdm
from Bio import Entrez, SeqIO
from tqdm import tqdm
import pandas as pd
import pickle
from genes_ncbi_mus_musculus_proteincoding import GENEID2NT


# multiprocessing.set_start_method('spawn')
df = pd.DataFrame.from_dict(GENEID2NT, orient='index')


def retrieve_sequences(geneID_list):
    """ Retrieve the sequences of the genes and the accession number.
    Then store them in a dictionary.
    
    Args:
        geneID_list (np.array): list of geneID to retrieve the sequences.
        
    Returns:
        dict_sequences (dict): dictionary with the geneID as key and the
            sequence and the accession number as values.
    
    
    """
    
    dict_sequences = {}
    for geneID in tqdm(geneID_list):
        
        # temporary dictionary
        temp = {}
        
        # get the accession number
        handle = Entrez.esearch(db="nucleotide", term=f"gene_id:{geneID}")
        record = Entrez.read(handle)
        accession = record["IdList"][0]
        
        # fetch the sequence
        handle = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", retmode="text")
        record = SeqIO.read(handle, "fasta")
        
        # store the informtions in a dictionary
        temp['accession'] = accession
        temp['sequence'] = str(record.seq)
        
        dict_sequences[geneID] = temp
        
        with open('./proteins/go_term/dict_retrieved_genes_checkpoint56.pkl', 'wb') as f:
            pickle.dump(dict_sequences, f)
        
    return dict_sequences
        
from Bio import Entrez, SeqIO

dictionary_sequences = retrieve_sequences(df['GeneID'].values[56:])
# problematic I try with the multiprocessing library

In [ ]:
import multiprocessing
from Bio import Entrez, SeqIO
from tqdm import tqdm
import pandas as pd
from genes_ncbi_mus_musculus_proteincoding import GENEID2NT


# multiprocessing.set_start_method('spawn')
df = pd.DataFrame.from_dict(GENEID2NT, orient='index')



def retrieve_sequences(geneID):
    """ Retrieve the sequences of the genes and the accession number.
    Then store them in a dictionary.
    
    Args:
        geneID_list (np.array): list of geneID to retrieve the sequences.
        
    Returns:
        dict_sequences (dict): dictionary with the geneID as key and the
            sequence and the accession number as values.
    
    
    """
    # temporary dictionary
    temp = {}
    
    # get the accession number
    handle = Entrez.esearch(db="nucleotide", term=f"gene_id:{geneID}")
    record = Entrez.read(handle)
    accession = record["IdList"][0]
    
    # fetch the sequence
    handle = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", retmode="text")
    record = SeqIO.read(handle, "fasta")
    
    # store the informtions in a dictionary
    temp['accession'] = accession
    temp['sequence'] = str(record.seq)
                
    return (geneID, temp)


# Set up a multiprocessing pool with 4 processes
pool = multiprocessing.Pool(processes=4)

# Use the multiprocessing pool to retrieve the sequences in parallel
results = pool.imap(retrieve_sequences, df['GeneID'].values)

# Close the multiprocessing pool
pool.close()
pool.join()

# Create a dictionary from the results
dict_sequences = {}
for result in results:
    dict_sequences[result[0]] = result[1]



In [2]:
# import urllib.request

# # Define the URL to download the human gene sequences from the NCBI RefSeq database
# url = "ftp://ftp.ncbi.nlm.nih.gov/refseq/H_sapiens/annotation/GRCh38_latest/refseq_identifiers/GRCh38_latest_genomic.fna.gz"

# # Download the file and save it to disk
# urllib.request.urlretrieve(url, "human_gene_sequences.fa.gz")

# curl -o human_gene_sequences.fa.gz ftp://ftp.ncbi.nlm.nih.gov/refseq/H_sapiens/annotation/GRCh38_latest/refseq_identifiers/GRCh38_latest_genomic.fna.gz

# with open("GRCh38_latest_genomic.fna", "r") as f:
#     for line in f:
#         print(line)

import pandas as pd
from genes_ncbi_mus_musculus_proteincoding import GENEID2NT


# multiprocessing.set_start_method('spawn')
df = pd.DataFrame.from_dict(GENEID2NT, orient='index')


,tax_id,Org_name,GeneID,CurrentID,Status,Symbol,Aliases,description,other_designations,map_location,chromosome,genomic_nucleotide_accession_version,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,orientation,exon_count,OMIM,no_hdr0
1,9606,Homo sapiens,1,0,live,A1BG,"[A1B, ABG, GAB, HYST2477]",alpha-1-B glycoprotein,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...,19q13.43,19,NC_000019.10,58345183,58353492,minus,8,[138670],
2,9606,Homo sapiens,2,0,live,A2M,"[A2MD, CPAMD5, FWP007, S863-7]",alpha-2-macroglobulin,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...,12p13.31,12,NC_000012.12,9067708,9116229,minus,37,[103950],
9,9606,Homo sapiens,9,0,live,NAT1,"[AAC1, MNAT, NAT-1, NATI]",N-acetyltransferase 1,arylamine N-acetyltransferase 1|N-acetyltransf...,8p22,8,NC_000008.11,18170467,18223689,plus,11,[108345],
10,9606,Homo sapiens,10,0,live,NAT2,"[AAC2, NAT-2, PNAT]",N-acetyltransferase 2,arylamine N-acetyltransferase 2|N-acetyltransf...,8p22,8,NC_000008.11,18386301,18401218,plus,4,[612182],
12,9606,Homo sapiens,12,0,live,SERPINA3,"[AACT, ACT, GIG24, GIG25]",serpin family A member 3,alpha-1-antichymotrypsin|cell growth-inhibitin...,14q32.13,14,NC_000014.9,94612391,94624053,plus,7,[107280],
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128966782,9606,Homo sapiens,128966782,0,live,LOC128966782,[],uncharacterized LOC128966782,uncharacterized protein LOC128966782,,13,,,,,,[],
128966783,9606,Homo sapiens,128966783,0,live,LOC128966783,[],uncharacterized LOC128966783,,,13,,,,,,[],
128966784,9606,Homo sapiens,128966784,0,live,LOC128966784,[],pollen-specific leucine-rich repeat extensin-l...,pollen-specific leucine-rich repeat extensin-l...,,14,,,,,,[],
128966785,9606,Homo sapiens,128966785,0,live,LOC128966785,[],protein mago nashi homolog,protein mago nashi homolog,,14,,,,,,[],


In [15]:
# since was impossible to download the annotations
# from the api I downloaded them directly from  
# uniref website
file = 'uniprot-annotations.tsv'
with open(file, 'rb') as fd:
    gzip_fd = gzip.GzipFile(fileobj=fd)
    destinations = pd.read_csv(gzip_fd, delimiter='\t', on_bad_lines='skip')

In [24]:
# for those genes I found the sequence 
print('Number of gene which match:', (sum([1 for i in df['GeneID'].values if len(destinations[destinations['GeneID']==f'{i};']) > 0])))

Number of gene which match: 18531


In [26]:
print('Number of gene which match:', (sum([1 for i in df['Symbol'].values if len(destinations[destinations['Gene Names (primary)']==i]) > 0])))

Number of gene which match: 19068


In [25]:
destinations.head()

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Gene Ontology (biological process),Gene Ontology (cellular component),Gene Ontology (molecular function),...,Organism (ID),Gene Names (primary),Sequence,Keyword ID,PubMed ID,RefSeq,EMBL,STRING,GeneID,Pfam
0,A0A087X1C5,reviewed,CP2D7_HUMAN,Putative cytochrome P450 2D7 (EC 1.14.14.1),CYP2D7,Homo sapiens (Human),515,arachidonic acid metabolic process [GO:0019369...,cytoplasm [GO:0005737]; intracellular membrane...,aromatase activity [GO:0070330]; heme binding ...,...,9606,CYP2D7,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...,KW-0963; KW-0325; KW-0349; KW-0408; KW-0472; K...,15051713; 10591208; 16169517; 17494644; 18838503,NP_001335315.1;,AY220845;AC254562;,NaN,1564;,PF00067;
1,A0A0B4J2F0,reviewed,PIOS1_HUMAN,Protein PIGBOS1 (PIGB opposite strand protein 1),PIGBOS1,Homo sapiens (Human),54,regulation of endoplasmic reticulum unfolded p...,mitochondrial outer membrane [GO:0005741],NaN,...,9606,PIGBOS1,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLV...,KW-0903; KW-0472; KW-0496; KW-1000; KW-1185; K...,14702039; 16572171; 31653868,NP_001295350.1;NP_001295351.1;NP_001295352.1;,AB593170;AC018926;,NaN,101928527;,NaN
2,A0A0B4J2F2,reviewed,SIK1B_HUMAN,Putative serine/threonine-protein kinase SIK1B...,SIK1B,Homo sapiens (Human),783,intracellular signal transduction [GO:0035556]...,cytoplasm [GO:0005737]; nucleus [GO:0005634],ATP binding [GO:0005524]; magnesium ion bindin...,...,9606,SIK1B,MVIMSEFSADPAGQGQGQQKPLRVGFYDIERTLGKGNFAVVKLARH...,KW-0067; KW-0418; KW-0460; KW-0479; KW-0547; K...,10830953,NP_001307572.1;,CU639417;CH471079;,9606.ENSP00000482829;,102724428;,PF00069;
3,A0A0C5B5G6,reviewed,MOTSC_HUMAN,Mitochondrial-derived peptide MOTS-c (Mitochon...,MT-RNR1,Homo sapiens (Human),16,activation of protein kinase activity [GO:0032...,extracellular space [GO:0005615]; mitochondrio...,DNA binding [GO:0003677]; DNA-binding transcri...,...,9606,MT-RNR1,MRWQEMGYIFYPRKLR,KW-0238; KW-0496; KW-0539; KW-0892; KW-1185; K...,25738459; 7219534; 29096170; 29886458; 2998324...,NaN,KP715230;J01415;,NaN,NaN,NaN
4,A0A0K2S4Q6,reviewed,CD3CH_HUMAN,Protein CD300H (CD300 antigen-like family memb...,CD300H,Homo sapiens (Human),201,neutrophil chemotaxis [GO:0030593],extracellular region [GO:0005576]; plasma memb...,transmembrane signaling receptor activity [GO:...,...,9606,CD300H,MTQRAGAAMLPSALLLLCVPGCLTVSGPSTVMGAVGESLSVQCRYE...,KW-0025; KW-1015; KW-0325; KW-0393; KW-0472; K...,26221034; 16625196,NP_001311002.1 [A0A0K2S4Q6-1];NP_001311005.1 [...,LC013475;LC013476;AC079325;,NaN,100130520;,PF07686;


In [32]:
print('maximum lenght: ',max(destinations['Length'].values))
print('minimum lenght: ',min(destinations['Length'].values))
print('mean lenght: ', np.mean(destinations['Length'].values))
print('median lenght: ', np.median(destinations['Length'].values))
print('std lenght: ', np.std(destinations['Length'].values))

maximum lenght:  34350
minimum lenght:  2
mean lenght:  557.8200558194193
median lenght:  415.0
std lenght:  599.5678616364435
